In [ ]:
import timeit
import requests
import json
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
import json
import numpy as np
import csv
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
user='postgres'
passwd='postgres'
host='postgresql'
port='5432'
db='cartola'
url = 'postgresql://{user}:{passwd}@{host}:{port}/{db}'.format(user=user, passwd=passwd, host=host, port=port, db=db)
print(url)

#engine = create_engine(url)

In [ ]:
def create_database_session():
    #engine = create_engine(f'postgresql://{user}:{passwd}@{address}/{db_name}')
    engine = create_engine(url)
    return engine

In [ ]:
def erase_data(engine):
    engine.execute('TRUNCATE match')

In [ ]:
def create_table(engine):
    engine.execute('''
        CREATE TABLE IF NOT EXISTS match (
            match_id integer, 
            turn smallint,
            home_id smallint,
            visitor_id smallint,
            date varchar(20),
            timestamp bigint,
            local varchar(80),
            valid boolean,
            home_goal smallint,
            visitor_goal smallint
        )'''
    )

In [ ]:
def save_data(engine, dataframe):
    dataframe.to_sql('match', con=engine, index=False, if_exists='replace')

In [ ]:
def count_data(engine):
    return engine.execute('SELECT count(*) FROM match')

# Carregando dados dos jogos

In [ ]:
start = timeit.default_timer()

In [ ]:
home_page = requests.get('https://api.cartolafc.globo.com/partidas')
home_json = json.loads(home_page.content)
last_turn = home_json['rodada']
print(f'Rodada atual: {last_turn}')

In [ ]:
matches = []
for turn in range(1, last_turn):
    turn_page = requests.get(f'https://api.cartolafc.globo.com/partidas/{turn}')
    turn_json = json.loads(turn_page.content)
    turn_matches = turn_json['partidas']
    for match_data in turn_matches:
        match = {
            'match_id': match_data['partida_id'],
            'turn': turn,
            'home_id': match_data['clube_casa_id'],
            'visitor_id': match_data['clube_visitante_id'],
            'date': match_data['partida_data'],
            'timestamp': match_data['timestamp'],
            'local': match_data['local'],
            'valid': match_data['valida'],
            'home_goal': match_data['placar_oficial_mandante'],
            'visitor_goal': match_data['placar_oficial_visitante'],
            
        }
        matches.append(match)

In [ ]:
df_matches = pd.DataFrame(matches)
df_matches.shape

In [ ]:
engine = create_database_session()
#create_table(engine)
erase_data(engine)
save_data(engine, df_matches)
count_data(engine).fetchall()

In [ ]:
end = timeit.default_timer()
print ('Duração: %.f segundo(os)' % ((end - start) ))

# Carregando dados de jogadores

In [ ]:
# Obtendo os dados pela API
def getDadosRodada (rodada):
    url = 'https://api.cartolafc.globo.com/atletas/pontuados/{}'.format(rodada)
    r = requests.get(url)
    json_dados_rodada = r.json()
    numero_rodada = json_dados_rodada['rodada']
    total_atletas_rodada = json_dados_rodada['total_atletas']
    df_atletas = pd.DataFrame.from_dict(json_dados_rodada['atletas'], orient='index')
    df_atletas['id_atleta'] = df_atletas.index
    json_struct = json.loads(df_atletas.to_json(orient="records"))    
    #df_atletas_flat = pd.io.json.json_normalize(json_struct) #use pd.io.json
    df_atletas_flat = pd.json_normalize(json_struct) #use pd.io.json
    df_atletas_flat.fillna(0, inplace=True)
    return df_atletas_flat, numero_rodada, total_atletas_rodada

In [ ]:
#verificacoes
def verificaDados(df_atletas_flat, numero_rodada, total_atletas_rodada):
    #verifica se o numero de atletas esta ok
    if df_atletas_flat.shape[0] != total_atletas_rodada:
        print('Erro no numero de atletas da rodada! apurado:{}, indicado:{}'.format(df_atletas_flat.shape[0], total_atletas_rodada))
        return False
    else:
        print('Verificacao numero de atletas da rodada ok! apurado:{}, indicado:{}'.format(df_atletas_flat.shape[0], total_atletas_rodada))

    #verifica se possui todos os scouts
    for i in scouts_possiveis:
        if i not in df_atletas_flat.columns:
            print('Coluna {} nao encontrada. Adicionando'.format(i))
            df_atletas_flat[i] = 0

    #verifica se a pontuacao dos atletas esta ok
    df_atletas_flat['pontuacao_calculada']=\
        df_atletas_flat['scout.G']*8.0+\
        df_atletas_flat['scout.A']*5.0+\
        df_atletas_flat['scout.FT']*3.0+\
        df_atletas_flat['scout.FD']*1.2+\
        df_atletas_flat['scout.FF']*0.8+\
        df_atletas_flat['scout.FS']*0.5+\
        df_atletas_flat['scout.PS']*1.0+\
        df_atletas_flat['scout.PP']*-4.0+\
        df_atletas_flat['scout.I']*-0.5+\
        df_atletas_flat['scout.PI']*-0.1+\
        df_atletas_flat['scout.DP']*7.0+\
        df_atletas_flat['scout.SG']*5.0+\
        df_atletas_flat['scout.DE']*1.0+\
        df_atletas_flat['scout.DS']*1.0+\
        df_atletas_flat['scout.CA']*-2.0+\
        df_atletas_flat['scout.GS']*-1.0+\
        df_atletas_flat['scout.FC']*-0.5+\
        df_atletas_flat['scout.PC']*-1.0+\
        df_atletas_flat['scout.GC']*-5.0+\
        df_atletas_flat['scout.CV']*-5.0

    df_atletas_flat['diff_pontuacao'] = abs(df_atletas_flat['pontuacao_calculada']-df_atletas_flat['pontuacao'])

    numero_atletas_errado = df_atletas_flat[(df_atletas_flat['posicao_id']!=6) & (df_atletas_flat['diff_pontuacao'] > 0.001)].shape[0]

    if numero_atletas_errado > 0:
        print('Erro na pontuacao de alguns atletas. Numero de atletas com problema:{}'.format(numero_tletas_errado))
        return False
    else:
        print('Pontuacao dos atletas OK')

    df_atletas_flat['rodada'] = numero_rodada

    # exclui colunas scout, pontuacao_calculada e diff_pontuacao
    df_atletas_flat = df_atletas_flat[['apelido', 'foto', 'pontuacao', 'posicao_id', 'clube_id',\
           'entrou_em_campo', 'id_atleta', 'scout.FC', 'scout.FS', 'scout.PI',\
           'scout.FF', 'scout.A', 'scout.DS', 'scout.DE', 'scout.GS', 'scout.FD',\
           'scout.SG', 'scout.CA', 'scout.FT', 'scout.G', 'scout.PC',\
           'scout.PS', 'scout.PP', 'scout.I', 'scout.DP', 'scout.GC', 'scout.CV',\
           'rodada']]
    return True, df_atletas_flat

In [ ]:
scouts_possiveis = ['scout.FC', 'scout.FS', 'scout.PI',\
       'scout.FF', 'scout.A', 'scout.DS', 'scout.DE', 'scout.GS', 'scout.FD',\
       'scout.SG', 'scout.CA', 'scout.FT', 'scout.G', 'scout.PC',\
       'scout.PS', 'scout.PP', 'scout.I', 'scout.DP', 'scout.GC', 'scout.CV']

In [ ]:
df_all = pd.DataFrame()

In [ ]:
for num_rodada in range(1, last_turn):
    print('Processando rodada={}'.format(num_rodada))
    df_atletas_flat, numero_rodada, total_atletas_rodada = getDadosRodada(num_rodada)
    flagVerificao, df_atletas_flat = verificaDados(df_atletas_flat, numero_rodada, total_atletas_rodada)
    if not flagVerificao:
        print('================= Dados nao confiaveis ===================')
    else:
        df_all = df_all.append(df_atletas_flat, ignore_index = True)

In [ ]:
df_all.shape

In [ ]:
df_scouts = df_all[['apelido', 'pontuacao', 'posicao_id', 'clube_id',\
       'entrou_em_campo', 'id_atleta', 'scout.FC', 'scout.FS', 'scout.PI',\
       'scout.FF', 'scout.A', 'scout.DS', 'scout.DE', 'scout.GS', 'scout.FD',\
       'scout.SG', 'scout.CA', 'scout.FT', 'scout.G', 'scout.PC', 'scout.PS',\
       'scout.PP', 'scout.I', 'scout.DP', 'scout.GC', 'scout.CV', 'rodada']]

In [ ]:
df_scouts.columns = ['player_name', 'points', 'id_position', 'id_club',\
                    'has_played', 'id_player', 'scout_fc', 'scout_fs', 'scout_pi',\
       'scout_ff', 'scout_a', 'scout_ds', 'scout_de', 'scout_gs', 'scout_fd',\
       'scout_sg', 'scout_ca', 'scout_ft', 'scout_g', 'scout_pc', 'scout_ps',\
       'scout_pp', 'scout_i', 'scout_dp', 'scout_gc', 'scout_cv', 'round']

In [ ]:
#df_scouts.to_sql('scouts', engine, if_exists='append', index=False)
df_scouts.to_sql('scouts', engine, if_exists='replace', index=False)

# Analises

In [ ]:
query = 'select * from match;'
df_match = pd.read_sql(query, con=engine)
df_match.shape

In [ ]:
df_scouts = pd.read_sql('select * from scouts', con=engine)
df_scouts.shape

In [ ]:
df_club = pd.read_sql('select * from club', con=engine)
df_club.shape

In [ ]:
df_match.head()

In [ ]:
df_scouts.head()

In [ ]:
df_club.head()

In [ ]:
df_scouts['round'].value_counts().sort_index()

# Analise dos Jogos

In [ ]:
df_match['diff_goals'] = df_match['home_goal']-df_match['visitor_goal']

In [ ]:
def vencedor(diff):
    if diff > 0:
        return 'CASA'
    elif diff == 0:
        return 'EMPATE'
    else:
        return 'VISITANTE'

In [ ]:
df_match['vencedor'] = df_match['diff_goals'].map(vencedor)

In [ ]:
df_match.head()

In [ ]:
df_match.shape

In [ ]:
df_match['datetime'] = pd.to_datetime(df_match['date'])

In [ ]:
df_match.dtypes

In [ ]:
df_match['vencedor'].value_counts()

In [ ]:
numero_jogos = len(df_match)
numero_jogos_vencedor_casa = len(df_match[df_match['vencedor']=='CASA'])
numero_jogos_vencedor_fora = len(df_match[df_match['vencedor']=='VISITANTE'])
numero_jogos_empate = len(df_match[df_match['vencedor']=='EMPATE'])

In [ ]:
if (numero_jogos_vencedor_casa+numero_jogos_vencedor_fora+numero_jogos_empate) != numero_jogos:
    print('Inconsistencia encontrada!!!!!!!!!!!!!!!!!!!!!!!!!!')
else:
    print('Dados consistentes')

In [ ]:
numero_jogos

In [ ]:
print('Percentual de vitoria CASA={}'.format(numero_jogos_vencedor_casa/numero_jogos))
print('Percentual de vitoria EMPATE={}'.format(numero_jogos_empate/numero_jogos))
print('Percentual de vitoria FORA={}'.format(numero_jogos_vencedor_fora/numero_jogos))

In [ ]:
df_match_before = df_match[df_match['datetime'] <= '2021-10-01 00:00:00']

In [ ]:
df_match_before['vencedor'].value_counts()

In [ ]:
numero_jogos = len(df_match_before)
numero_jogos_vencedor_casa = len(df_match_before[df_match_before['vencedor']=='CASA'])
numero_jogos_vencedor_fora = len(df_match_before[df_match_before['vencedor']=='VISITANTE'])
numero_jogos_empate = len(df_match_before[df_match_before['vencedor']=='EMPATE'])

In [ ]:
print('Percentual de vitoria - S/ TORCIDA')
print('Percentual de vitoria CASA={}'.format(numero_jogos_vencedor_casa/numero_jogos))
print('Percentual de vitoria EMPATE={}'.format(numero_jogos_empate/numero_jogos))
print('Percentual de vitoria FORA={}'.format(numero_jogos_vencedor_fora/numero_jogos))

In [ ]:
df_match_after = df_match[df_match['datetime'] > '2021-10-01 00:00:00']
df_match_after['vencedor'].value_counts()

In [ ]:
numero_jogos = len(df_match_after)
numero_jogos_vencedor_casa = len(df_match_after[df_match_after['vencedor']=='CASA'])
numero_jogos_vencedor_fora = len(df_match_after[df_match_after['vencedor']=='VISITANTE'])
numero_jogos_empate = len(df_match_after[df_match_after['vencedor']=='EMPATE'])

In [ ]:
print('Percentual de vitoria - C/ TORCIDA')
print('Percentual de vitoria CASA={}'.format(numero_jogos_vencedor_casa/numero_jogos))
print('Percentual de vitoria EMPATE={}'.format(numero_jogos_empate/numero_jogos))
print('Percentual de vitoria FORA={}'.format(numero_jogos_vencedor_fora/numero_jogos))

In [ ]:
df_match[['home_goal', 'visitor_goal']].describe()

# Análise dos Jogadores

In [ ]:
df_scouts.head()

In [ ]:
df_match.head()

In [ ]:
df_match.columns

In [ ]:
df_scouts.columns

In [ ]:
df_match.columns = ['match_id', 'round', 'id_club', 'visitor_id', 'date', 'timestamp',\
       'local', 'valid', 'home_goal', 'visitor_goal', 'diff_goals', 'vencedor',\
       'datetime']

In [ ]:
def is_home(home_id, id_club):
    if home_id == id_club:
        return 1
    else:
        return 0

In [ ]:
df_scouts = pd.merge(df_scouts, df_match, how='left', on=['round', 'id_club'])

In [ ]:
df_match.columns = ['match_id', 'round', 'home_id', 'id_club', 'date', 'timestamp',\
       'local', 'valid', 'home_goal', 'visitor_goal', 'diff_goals', 'vencedor',\
       'datetime']

In [ ]:
df_scouts = pd.merge(df_scouts, df_match, how='left', on=['round', 'id_club'])

In [ ]:
df_scouts.head()

In [ ]:
df_scouts.shape

In [ ]:
df_scouts.columns

In [ ]:
df_scouts['home_id'].fillna(df_scouts['id_club'], inplace= True)
df_scouts['visitor_id'].fillna(df_scouts['id_club'], inplace= True)
df_scouts['home_goal_x'].fillna(df_scouts['home_goal_y'], inplace= True)
df_scouts['visitor_goal_x'].fillna(df_scouts['visitor_goal_y'], inplace= True)
#df_scouts['home_x'].fillna(df_scouts['home_y'], inplace= True)
#df_scouts['visitor_x'].fillna(df_scouts['visitor_y'], inplace= True)
df_scouts['home_goal'] = df_scouts['home_goal_x']
df_scouts['visitor_goal'] = df_scouts['visitor_goal_x'] 

#df_scouts['home'] = df_scouts['home_x']
#df_scouts['visitor'] = df_scouts['visitor_x']
df_scouts = df_scouts[['player_name', 'points', 'id_position', 'id_club', 'has_played',\
       'id_player', 'scout_fc', 'scout_fs', 'scout_pi', 'scout_ff', 'scout_a',\
       'scout_ds', 'scout_de', 'scout_gs', 'scout_fd', 'scout_sg', 'scout_ca',\
       'scout_ft', 'scout_g', 'scout_pc', 'scout_ps', 'scout_pp', 'scout_i',\
       'scout_dp', 'scout_gc', 'scout_cv', 'round', 'visitor_id',\
       'home_id',\
       'home_goal',\
       'visitor_goal']]

In [ ]:
df_scouts.head(10)

In [ ]:
df_scouts['CASA'] = df_scouts.apply(lambda row : is_home(row['home_id'], row['id_club']), axis = 1)

In [ ]:
df_clube = df_club[['id', 'name']]
df_clube.columns = ['home_id', 'time_casa']

In [ ]:
df_scouts = pd.merge(df_scouts, df_clube, how='left', on=['home_id'])

In [ ]:
df_scouts.head()

In [ ]:
df_clube.columns = ['visitor_id', 'time_visitante']
df_scouts = pd.merge(df_scouts, df_clube, how='left', on=['visitor_id'])

In [ ]:
df_clube.columns = ['id_club', 'time']
df_scouts = pd.merge(df_scouts, df_clube, how='left', on=['id_club'])

In [ ]:
df_scouts.head()

In [ ]:
df_pontuacao_casa = df_scouts[df_scouts['CASA']==1].groupby(['id_player','time', 'player_name', 'id_club', 'id_position']).agg({'points':'sum', 'has_played':'sum'}).reset_index()
df_pontuacao_fora = df_scouts[df_scouts['CASA']==0].groupby(['id_player', 'player_name', 'id_club', 'id_position']).agg({'points':'sum', 'has_played':'sum'}).reset_index()

In [ ]:
df_pontuacao_casa.columns=['id_player','time','player_name', 'id_club', 'id_position', 'points_casa', 'has_played_casa']
df_pontuacao_fora.columns=['id_player','player_name', 'id_club', 'id_position', 'points_fora', 'has_played_fora']

In [ ]:
df_pontuacao = pd.merge(df_pontuacao_casa, df_pontuacao_fora, how='outer', on=['id_player','player_name','id_club','id_position'])

In [ ]:
df_pontuacao.fillna(0, inplace=True)

In [ ]:
df_pontuacao['total_pontos']=df_pontuacao['points_fora']+df_pontuacao['points_casa']
df_pontuacao['total_jogos']=df_pontuacao['has_played_casa']+df_pontuacao['has_played_fora']

In [ ]:
df_pontuacao.head()

In [ ]:
df_pontuacao.sort_values(by='total_pontos', ascending=False).head(20)

In [ ]:
df_pontuacao[df_pontuacao['id_position']==5].sort_values(by='total_pontos', ascending=False).head(20)

In [ ]:
df_pontuacao[df_pontuacao['id_position']==2].sort_values(by='total_pontos', ascending=False).head(10)

In [ ]:
df_pontuacao[df_pontuacao['id_position']==3].sort_values(by='total_pontos', ascending=False).head(10)

In [ ]:
df_pontuacao[df_pontuacao['id_position']==6].sort_values(by='total_pontos', ascending=False).head(10)

In [ ]:
df_pontuacao.sort_values(by='points_casa', ascending=False).head(20)

In [ ]:
df_pontuacao.sort_values(by='points_fora', ascending=False).head(10)

### Histogramas de pontos dos Jogadores

In [ ]:
df_scouts.head()

In [ ]:
## Adicionar coluna Adversário.
## Montar histograma de times que cederam mais pontos.
## Montar histograma de times que cederam mais pontos por posição de jogador.
## Criar coluna dos jogadores que estão pontuando em P75

In [ ]:
df_scouts.shape

In [ ]:
df_scouts.columns

In [ ]:
df_scouts['points'].hist(bins=range(-10, 35, 1), figsize=(15, 6))

In [ ]:
df_scouts[df_scouts['has_played'] == True].shape

In [ ]:
df_scouts[df_scouts['has_played'] == True]['points'].hist(bins=range(-10, 35, 1), figsize=(15, 6))

In [ ]:
df_pontuacao_por_posicao = df_scouts[df_scouts['has_played'] == True]

In [ ]:
df_pontuacao_por_posicao.head()

In [ ]:
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 5].shape

In [ ]:
## Atacantes
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 5]['points'].hist(bins=range(-10, 35, 1), figsize=(15, 6))

In [ ]:
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 5]['points'].describe()

In [ ]:
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 4].shape

In [ ]:
## Meia
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 4]['points'].hist(bins=range(-10, 35, 1), figsize=(15, 6))

In [ ]:
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 4]['points'].describe()

In [ ]:
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 3].shape

In [ ]:
## Zagueiro
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 3]['points'].hist(bins=range(-10, 35, 1), figsize=(15, 6))

In [ ]:
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 3]['points'].describe()

In [ ]:
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 2].shape

In [ ]:
## Lateral
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 2]['points'].hist(bins=range(-10, 35, 1), figsize=(15, 6))

In [ ]:
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 2]['points'].describe()

In [ ]:
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 1].shape

In [ ]:
## Goleiro
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 1]['points'].hist(bins=range(-10, 35, 1), figsize=(15, 6))

In [ ]:
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 1]['points'].describe()

In [ ]:
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 6].shape

In [ ]:
## Técnico
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 6]['points'].hist(bins=range(-10, 35, 1), figsize=(15, 6))

In [ ]:
df_pontuacao_por_posicao[df_pontuacao_por_posicao['id_position'] == 6]['points'].describe()

### Analise de Tendência

Desempenho dos jogadores

In [ ]:
df_tendencia_scouts = df_scouts.set_index('round')
df_tendencia_scouts.head()

In [ ]:
df_tendencia_scouts = df_tendencia_scouts[df_tendencia_scouts['has_played'] == True]

In [ ]:
df_tendencia_scouts = df_tendencia_scouts[['player_name', 'points']]
df_tendencia_scouts.head()

### Aplicação de tendência

Utilizando os rounds como medida temporal podemos perceber as fases do jogador.

Ao utilizar uma média móvel (que se desloca no 'tempo/rounds') a curva da tendência fica suavizada e temos uma percepção melhor do que está acontecendo.
Alterando o parâmetro da função rolling podemos suavizar mais ainda a curva aumentando seu valor.

In [ ]:
jogador = 'Hulk'
janela = 10
fig, ax = plt.subplots()
df_tendencia_scouts[df_tendencia_scouts['player_name'] == jogador].plot(figsize=(15,6), ax=ax)
df_tendencia_scouts[df_tendencia_scouts['player_name'] == jogador].rolling(janela).mean().plot(figsize=(15,6), ax=ax)
ax.legend(["Pontos", "Tendência Média"]);